### **Instalando e importando as bibliotecas necessárias**

In [12]:
%pip install PyOpenGL
%pip install glfw
%pip install pyglm

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\kasat\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm

### **Inicializando a janela**

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(720, 600, "Avião em voo", None, None)

if (window == None):
    print("Failed to create GLFW window")
    glfwTerminate()
    
glfw.make_context_current(window)

### **Shaders**
Defininindo os programas Vertex e Fragment Shaders e requisitando slot para a GPU para eles. Após isso, associamos nosso código-fonte aos slots associados.

In [3]:
vertex_code = """
        attribute vec2 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,0.0,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

Compilando nossos programas e associando ao programa principal

In [4]:
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem e Preparação de Dados para a GPU

Build do programa tornando ele default

In [5]:
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
glUseProgram(program)

Modelagem dos objetos em cena

In [6]:
vertices = np.array([
    # Cabeça
    (0.5, 0.2, 0.0),   # 0 
    (0.0, -0.8, 0.0),  # 1
    (-0.5, 0.2, 0.0),  # 2

    # Corpo
    (0.5, -0.5, 0.0),  # 3
    (-0.5, -0.5, 0.0), # 4
    (0.3, -1.5, 0.0),  # 5
    (-0.3, -1.5, 0.0), # 6
    (0.3, -2.0, 0.0),  # 7
    (-0.3, -2.0, 0.0), # 8

    # Asas
    (2.0, -0.6, 0.0),  # 9
    (-2.0, -0.6, 0.0), # 10
    (0.0, -1.5, 0.0),  # 11
    (2.5, -2.0, 0.0),  # 12
    (-2.5, -2.0, 0.0), # 13
    (0.5, -0.6, 0.0),  # 14
    (-0.5, -0.6, 0.0), # 15

    # Rabo
    (0.7, -2.8, 0.0),  # 16
    (-0.7, -2.8, 0.0), # 17
    
], dtype=np.float32)

indices = np.array([
    # Cabeça
    0, 1, 2,

    # Corpo
    3, 4, 5,
    4, 5, 6,
    5, 6, 7,
    6, 7, 8,

    # Asas
    9, 10, 11,
    12, 9, 14,
    13, 10, 15,

    # Rabo
    7, 8, 16,
    16, 17, 8,
], dtype=np.uint32)


Configuração VAO/VBO/EBO

In [7]:
VAO = glGenVertexArrays(1)
glBindVertexArray(VAO)

VBO = glGenBuffers(1)
glBindBuffer(GL_ARRAY_BUFFER, VBO)
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)

EBO = glGenBuffers(1)
glBindBuffer(GL_ELEMENT_ARRAY_BUFFER, EBO)
glBufferData(GL_ELEMENT_ARRAY_BUFFER, indices.nbytes, indices, GL_STATIC_DRAW)

Associando variáveis do programa GLSL (Vertex Shader) com nossos dados

In [8]:
glVertexAttribPointer(0, 2, GL_FLOAT, False, 12, ctypes.c_void_p(0))
glEnableVertexAttribArray(0)

Definindo cores do projeto (mudar pra cá dps)

### **Capturando eventos de teclado e modificando variáveis para a matriz de transformação**

In [9]:
fator_escala = 0.5

def key_event(window,key,scancode,action,mods):
    global t_x, t_y, angulo, escala
    
    if key == 265: t_y += 0.01 #cima
    if key == 264: t_y -= 0.01 #baixo
    if key == 263: t_x -= 0.01 #esquerda
    if key == 262: t_x += 0.01 #direita

    if key == 65:  angulo += 0.05 # gira para a direita
    if key == 83:  angulo -= 0.05 # gira para a esquerda

    if key == 90: escala += 0.05 * fator_escala # aumenta escala 
    if key == 88: escala -= 0.05 * fator_escala # diminui escala

    if key == 82: 
        t_x = 0
        t_y = 0
        angulo = 0
        escala = 1


    
glfw.set_key_callback(window,key_event)

### **Funções Principais da Janela**

In [10]:
glfw.show_window(window)

In [11]:
import math

angulo = 0
t_x = 0.0
t_y = 0.0
escala = 0.3


def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

# Matriz de projeção ortográfica (mapeia coordenadas 3D para a tela)
glEnable(GL_BLEND);
glBlendFunc(GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA);
glClearColor(0.53, 0.81, 0.92, 1.0)  # Cor de céu
loc_color = glGetUniformLocation(program, "color")

# Loop principal
while not glfw.window_should_close(window):
    glClear(GL_COLOR_BUFFER_BIT)

    c = math.cos(angulo)
    s = math.sin(angulo)

    mat_translation = np.array([
        1, 0, 0, t_x,
        0, 1, 0, t_y,
        0, 0, 1, 0,
        0, 0, 0, 1
    ], np.float32)

    # Configura transformações
    mat_rotation = np.array([       c,   -s,  0.0, 0.0, 
                                    s,    c,  0.0, 0.0, 
                                    0.0, 0.0, 1.0, 0.0, 
                                    0.0, 0.0, 0.0, 1.0], np.float32)
    
    mat_scale = np.array([
        escala, 0, 0, 0,
        0, escala, 0, 0,
        0, 0, 1, 0,
        0, 0, 0, 1
    ], np.float32)

    loc = glGetUniformLocation(program, "mat_transformation")
    
    mat_partial = multiplica_matriz(mat_rotation, mat_translation)
    mat_definitive = multiplica_matriz(mat_partial, mat_scale)
    glUniformMatrix4fv(loc, 1, GL_TRUE, mat_definitive)

    # Desenha o pássaro
    glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0) 
    glDrawElements(GL_TRIANGLES, len(indices), GL_UNSIGNED_INT, None)

    # Descomente as duas linhas abaixo para conseguir ver as arestas
    # glUniform4f(loc_color, 0.0, 0.0, 0.0, 1.0)  # Cor preta
    # glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)    # Modo wireframe
    
    glfw.swap_buffers(window)
    glfw.poll_events()

glfw.terminate()